In [27]:
import numpy as np
import numpy.linalg as lin
from matplotlib import pyplot as plt
import math

In [32]:
# Make grid space
def xgrid(L,n):
    dx=L/n
    grd = np.linspace(-L/2., L/2., num = n, endpoint=False) # Make grid
    grd = np.array(grd, dtype = np.float64)
    return grd

def plot_grid(L,n, grd):
    dx=L/n
    strline = ""
    if "float" in str(grd.dtype):
        tmp_grd = grd
    if "complex" in str(grd.dtype):
        tmp_grd = grd.real**2 + grd.imag**2
    for i in range(n):
        strline += "  %.5e " % tmp_grd[i]
    strline += "  %.6e  \n" % tmp_grd[0]
    return strline

# Make wave fucntion (also construct it's operator)
def wave(L, n, l, dt):
    grd=np.zeros(n, np.complex64)
    dx=np.float64(L/n)
    
    if pot_shape == 5 :
        for i in range(n):                                   # wave packet initialize (gaussian)
            if (i > (n*4)//10 and i < (n*6)//10):
                grd[i] = np.exp(-(i*dx-0.5*n*dx)**2/sigma)
            else:
                grd[i] = 0. + 0.j
        grd /= lin.norm(grd)                       # Fix to normalize
        for i in range(n):
            grd[i] = grd[i]*np.exp(1j*k*(i*dx-0.5*n*dx))  #Wave packet

    else:
        for i in range(n):                                   # wave packet initialize (gaussian)
            if (i > n*0/10 and i < n*4/10):
                grd[i] = np.exp(-(i*dx-0.3*n*dx)**2/sigma)
            else:
                grd[i] = 0. + 0.j
        grd /= lin.norm(grd)                       # Fix to normalize
        for i in range(n):
            grd[i] = grd[i]*np.exp(1j*k*(i*dx-0.3*n*dx))  #Wave packet
    
    return grd     


#Potential part
# Construct Potential Operator using Potential grid
def Potential(L,n):
    grd=np.zeros(n)
    grd[0] = 100000000
    grd[1] = 100000000
    grd[n-1]= 100000000
    if pot_shape == 0:                                   #no potential
        for i in range(1, n):
            grd[i] = 0                           # Make potential

    if pot_shape == 1:                                   #Step potential
        for i in range(2, n-2):
            grd[i] = 0                           # Make potential
        for i in range((n//2),(n-2)):
            grd[i] = pot_height_eV              # eV unit
            grd[i] = grd[i]/27.211          # eV -> Har

    if pot_shape == 2:                                   #Potential barrier
        for i in range(2, n-2):
            grd[i] = 0                           # Make potential
        for i in range((5*n)//10,(5*n)//10+barrier_thickness*10):
            grd[i] = pot_height_eV                    # eV unit
            grd[i] = grd[i]/27.211          # eV -> Har

    if pot_shape == 3:                                   #Double barrier
        for i in range(2, n-2):
            grd[i] = 0                           # Make potential
        for i in range((45*n)//100-barrier_thickness*10,(45*n)//100):
            grd[i] = pot_height_eV                    # eV unit
            grd[i] = grd[i]/27.211          # eV -> Har
        for i in range((50*n)//100,(50*n)//100+barrier_thickness*10):
            grd[i] = pot_height_eV               # eV unit
            grd[i] = grd[i]/27.211          # eV -> Har

    if pot_shape == 4:                                   # Square well
        for i in range(2, n-2):
            grd[i] = 0                           # Make potential
        for i in range(2,(n*4)//10):
            grd[i]= pot_height_eV
            grd[i] = grd[i]/27.211          # eV -> Har
        for i in range((n*6)//10,n-2):
            grd[i]= pot_height_eV
            grd[i] = grd[i]/27.211          # eV -> Har

    if pot_shape == 5:                              #Harmonic well
        for i in range(1, n-1):
            x = L/(n-1)*i
            grd[i]=(i-n//2)**2/(n//2-1)**2*pot_height_eV/27.211

    return grd

# Define FDM points & coefficients (Here, 7-points FDM)
def fdmcoefficient(l):
    a=np.zeros((2*l+1,2*l+1))
    b=np.zeros(2*l+1)
    c=np.zeros(2*l+1)

    for i in range(0, 2*l+1):
        for j in range(0, 2*l+1):
            a[i,j]= (j-l)**i
    c[2]=2
    a = lin.inv(a)
    b= np.matmul(a, c)

    C=np.zeros((l+1))

    for i in range(0, l+1):
        C[i]=b[i+l]
    return C


###Laplacian and Hamiltonian
# Define Hamiltonian using Potential and Laplacian
def Laplacian(n, l, dx):
    C=fdmcoefficient(l)
    oprt=np.zeros((n,n))
    for i in range(n):
        for j in range(-l, l + 1, 1):
            k = i + j
            if (k >= n):
                k -= n
                oprt[i][k] = C[abs(j)] / (dx**2)
            elif (k<0):
                k += n
                oprt[i][k] = C[abs(j)] / (dx**2)
            else:
                oprt[i][k] = C[abs(j)] / (dx**2)
    return oprt


def Hamiltonian(L,n,l,dx):
    Hamiltonian = np.zeros((n,n))
    V = Potential(L, n)
    L = Laplacian(n, l, dx)                   # h- = 1, m_e = 1
    V_oprt=np.zeros((n,n))
    for i in range(0, n):
        V_oprt[i,i]=V[i]
    Hamiltonian = -L / 2. + V_oprt    # H = - (h-^2/2m) L + V
    return Hamiltonian


In [33]:
# Input parameter
Lx = 100                # Box Length (Angstrom = 10^-10 m)
ngx = 1001              # Number of grid points (spacing = L/(n-1))
nstep = 50              # Number of time steps * 0.1fs
ewave = 50         # Energy of packet (eV)
pot_shape = 2           # Shape of potential
                        # 0 = No potential
                        # 1 = Step potential
                        # 2 = Single wall
                        # 3 = Double wall
                        # 4 = Finite well (Packet starts at middle)
                        # 5 = Harmonic well

pot_height_eV = 10      # eV
barrier_thickness = 10  # Thickness of the barrier(Angstrom = 10^-10 m)
                        # Only for Potential_shape = 2 or 3!

dispersion_gaussian = 2.646  # Spatial dispersion(sigma) of gaussian wave packet (Angstrom = 10^-10 m)

lpacket = 0             # 0: Linear combination of eigenstates using all basis
                        # 1: Linear combination of eigenstates using ncombitates 

ncombistates = 1        # number of eigenstates in linear combination when lpacke=1


In [34]:
#Inputs & Unit conversion

k = (ewave*2/27.211)**0.5
sigma = dispersion_gaussian*1.88973
atomic_unit_of_time_to_fs = 2.418884326509E-2
angsL = Lx * 1.88973
l = 3
dt = 0.01/atomic_unit_of_time_to_fs
left = ngx//2
right = ngx//2
dx=np.float64(Lx/ngx)

#Make Hamiltonian & diagonalize -> get basis to construct the wave packet
H = Hamiltonian(Lx,ngx,l,dx)
E,phi = np.linalg.eigh(H)

# Make wave fucntion with basis (also construct it's operator)
## Initial wave packet (gaussian = f(x))
wave = wave(Lx, ngx, l,dt)

## Calculate coefficient & eigenfunction
c_n =(wave).dot(np.conjugate(phi))
Psi = c_n.dot(phi)
tt = np.linspace(0, (nstep-1)*dt,nstep)
z=np.zeros((ngx,ngx), dtype=complex)
Psi_t = np.zeros((nstep,ngx), dtype =complex)
xx=np.linspace(0, Lx, ngx)

f= open("wave.txt", 'w')
f.write("# t(fs) " )
for i in range(0, ngx):
    f.write('  %.6f  '%xx[i])
f.write('\n')

poten = Potential(Lx, ngx)

saveprob=[]

####Linear combination for Bound state###

bounce=np.zeros(ngx, complex)

for i in range(nstep):
    if(lpacket == 0):
        for j in range(ngx):
            z[:,j] = c_n[j]*(phi[:,j]*np.exp(-1j*E[j]*(tt[i])))
        Psi_t[i,:] = np.sum(z,1)
        
    if(lpacket == 1):
        for j in range(0, ncombistates):
            z[:,j] = c_n[j]*(phi[:,j]*np.exp(-1j*E[j]*(tt[i])))
        Psi_t[i,:] = np.sum(z,1)

    reflec = np.float64(0)
    trans = np.float64(0)

#counting
      
    prob = np.abs(Psi_t[i,:])**2
    for k in range(0, ngx):
            if k <= left :
                reflec += prob[k]
            if k > right :
                trans += prob[k]

# Save wave function as txt
    t=i*dt*atomic_unit_of_time_to_fs
    f.write('%.6f' %t)
    for k in range(0, ngx):
        f.write('  %.8f  ' %prob[k])
    f.write('\n')
    saveprob.append('%.6f    %.6f     %.6f\n' %(t,reflec,trans))
P=Potential(Lx,ngx)

f2 = open("Potential.txt",'w')
f2.write(plot_grid(Lx,ngx,P))
f2.close()

f3 = open("Probablity.txt",'w')
f3.write("# t(fs)    Reflection  Transmission\n")
f3.writelines(saveprob)
f3.close()

### Visualization

In [35]:

st=0

L=[]
file=open('wave.txt', 'r')

while (1):
    line=file.readline()

    try:escape=line.index('\n')
    except:escape=len(line)

    if line:
        L.append(line[0:escape])
    else:
        break

file.close()

L2=[]
file=open('Potential.txt', 'r')

while (1):
    line=file.readline()

    try:escape=line.index('\n')
    except:escape=len(line)

    if line:
        L2.append(line[0:escape])
    else:
        break

file.close()

tmp=L2[0].split()
a=np.size(tmp)
pot=np.zeros(a)
for i in range(0, a):
    pot[i]=tmp[i]

x = np.linspace(st, Lx, ngx-1)
nL = np.size(L)
store = np.zeros((nL-1,ngx))
a = np.max(pot[5:np.size(pot)-5])

for i in range(1, nL):
    tmp=L[i].split()
    for j in range(0, ngx):
        store[i-1,j]=np.float64(tmp[j])

for i in range(0,nstep):
    plt.plot(x, store[i*1,1:ngx], label= 'Time = %.6f fs' %(store[i*1,0]))
    if a != 0 :
        plt.plot(x, pot[0:ngx-1]/a*0.15, label='Potential = %.3f eV ' %(a*27.211))
    plt.xlim(0,Lx)
    plt.ylim(0,0.2)
    plt.legend()
    plt.yticks([], [])
    plt.xlabel('Box [Angstrom]')
#    plt.show()
    plt.savefig('case_%03d.png' %i)
    plt.clf()
    
############# save animation ############
from PIL import Image
import glob
 
# Create the frames
frames = []
imgs = glob.glob("*.png")
for i in imgs:
    new_frame = Image.open(i)
    frames.append(new_frame)

frames[0].save('movie.gif', format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=100, loop=0)

##### Erasing png files ##################
!del *.png

<Figure size 640x480 with 0 Axes>